# Skywalker algorithm against the lcls mirror systems

In [1]:
SIMULATION = False
ALIGNMENT = 'HOMS'
#ALIGNMENT = 'MEC'
#ALIGNMENT = 'MFX'
#ALIGNMENT = 'XCS'

alignment_goals = dict(HOMS=[220, 300],
                       MEC=[270],
                       MFX=[250],
                       XCS=[270, 270])
first_steps = 6 # Naive first step for the parameters search
tolerances = 10 # Tolerance of each target in Pixels
average = 75 # Number of shots to average over
timeout = 600 # Timeout for an alignment

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

In [2]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

In [3]:
"""
Parameters
----------
"""
#Create logging config with INFO to the console and DEBUG to the file
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename='./skywalker_debug.log',
                    filemode='a')
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(name)s - %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

from pswalker.examples  import patch_pims
from pswalker.skywalker import skywalker, lcls_RE
from pswalker.config import homs_system
from pswalker.plans import walk_to_pixel
from pswalker.watcher import Watcher
#Instantiate simulation
if not SIMULATION:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    mecy1 = system['mecy1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3
else:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', 'test_m1h_xy',
                              z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', 'test_m2h_xy',
                              x=0.0317324, z=101.843, alpha=0.0014)
    xrtm2 = mirror.OffsetMirror('test_xrtm2', 'test_xrtm2_xy',
                                 x=0.0317324, z=200, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    mecy1 = pim.PIM('test_mecy1', x=0.0317324, z=350,
             zero_outside_yag= not infinite_yag)
    mfxdg1 = mecy1
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)
    patch_pims([mecy1], mirrors=[xrtm2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise
    mecy1.centroid_noise = centroid_noise

alignment_yags = dict(HOMS=[y1, y2],
                      MEC=[mecy1],
                      MFX=[mfxdg1],
                      XCS=['pbty', 'xcssb1y'])
alignment_mots = dict(HOMS=[m1, m2],
                      MEC=[xrtm2],
                      MFX=[xrtm2],
                      XCS=['xrtm1', 'xrtm3'])

********************************************************************************
* WARNING: Running on real devices in Production...
********************************************************************************
Press any key to continue...gg


In [4]:
#Create Skywalker plan
yags = alignment_yags[ALIGNMENT]
mots = alignment_mots[ALIGNMENT]
goals = alignment_goals[ALIGNMENT]
if SIMULATION:
    det_rbv = 'detector_stats2_centroid_x'
else:
    det_rbv = 'detector_stats2_centroid_y'
mot_rbv = 'pitch'
plan = skywalker(yags, mots, det_rbv, mot_rbv,
        goals, first_steps=first_steps, tolerances=tolerances,
        averages=average, timeout=timeout, sim=SIMULATION, use_filters=not SIMULATION)

In [5]:
#Create RunEngine
if SIMULATION:
    RE = RunEngine()
else:
    RE = lcls_RE()
RE.record_interruptions = True
#Subscribe a Watcher instance
watcher = Watcher()
RE.msg_hook = watcher
RE.subscribe('all',  watcher)


0

In [6]:
logging.debug("Start of Run")
RE(plan)

At least one suspender has tripped. The plan will begin when all suspenders are ready. Justification:
    1. Signal GDET:FEE1:241:ENRC_avg = 0.6780610194062342 is below 1

Suspending... To get to the prompt, hit Ctrl-C twice to pause.
Suspender BeamEnergySuspendFloor(AvgSignal(name='GDET:FEE1:241:ENRC_avg', value=1.0020714391684624, timestamp=1502918524.6686897), sleep=5, pre_plan=<generator object PvSuspenderBase.__init__.<locals>.pre_plan at 0x7fc52933c150>, post_plan=<generator object PvSuspenderBase.__init__.<locals>.post_plan at 0x7fc52933c570>,tripped_message=) reports a return to nominal conditions. Will sleep for 5 seconds and then release suspension at 2017-08-16 14:22:09.
Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2017-08-16 14:22:04.
Justification for this suspension:
Signal GDET:FEE1:241:ENRC_avg = 0.99964275860474805 is below 1
Suspender BeamEnergySuspendFloor(AvgSignal(name='GDET:FEE1:241:ENRC_avg', value=1.0039696015764434, timestamp=15

pcdsdevices.epics.mirror - Waiting for m1h_pitch to finish move ...
pcdsdevices.epics.mirror - Waiting for m2h_pitch to finish move ...


Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2017-08-16 14:22:27.
Justification for this suspension:
Signal GDET:FEE1:241:ENRC_avg = 0.99859467324692786 is below 1
Suspender BeamEnergySuspendFloor(AvgSignal(name='GDET:FEE1:241:ENRC_avg', value=1.2941542261140579, timestamp=1502918547.3292625), sleep=5, pre_plan=<generator object PvSuspenderBase.__init__.<locals>.pre_plan at 0x7fc52933c150>, post_plan=<generator object PvSuspenderBase.__init__.<locals>.post_plan at 0x7fc52933c570>,tripped_message=) reports a return to nominal conditions. Will sleep for 5 seconds and then release suspension at 2017-08-16 14:22:32.
Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2017-08-16 14:22:27.
Justification for this suspension:
Signal GDET:FEE1:241:ENRC_avg = 0.99765996760148479 is below 1
Suspender BeamEnergySuspendFloor(AvgSignal(name='GDET:FEE1:241:ENRC_avg', value=1.2964444492096983, timestamp=1502918547.3347363), sleep=5, pre_plan=<

ophyd.utils.epics_pvs - Waiting for dg3_detector_image1_blocking_callbacks to be set from 'No' to 'Yes'...
ophyd.utils.epics_pvs - Waiting for dg3_detector_stats2_blocking_callbacks to be set from 'No' to 'Yes'...
ophyd.utils.epics_pvs - Waiting for dg3_detector_stats2_enable to be set from 0.0 to 1...


Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2017-08-16 14:22:38.
Justification for this suspension:
Signal GDET:FEE1:241:ENRC_avg = 0.998230798543841 is below 1
Suspender BeamEnergySuspendFloor(AvgSignal(name='GDET:FEE1:241:ENRC_avg', value=1.2724813467721492, timestamp=1502918558.656678), sleep=5, pre_plan=<generator object PvSuspenderBase.__init__.<locals>.pre_plan at 0x7fc52933c150>, post_plan=<generator object PvSuspenderBase.__init__.<locals>.post_plan at 0x7fc52933c570>,tripped_message=) reports a return to nominal conditions. Will sleep for 5 seconds and then release suspension at 2017-08-16 14:22:43.
Suspending....To get prompt hit Ctrl-C twice to pause.
Suspension occurred at 2017-08-16 14:22:38.
Justification for this suspension:
Signal GDET:FEE1:241:ENRC_avg = 0.99889292193581014 is below 1
Suspender BeamEnergySuspendFloor(AvgSignal(name='GDET:FEE1:241:ENRC_avg', value=1.2724813467721492, timestamp=1502918558.656678), sleep=5, pre_plan=<gene

ophyd.utils.epics_pvs - Waiting for dg3_detector_stats2_enable to be set from 1.0 to 0.0...
ophyd.utils.epics_pvs - Waiting for dg3_detector_stats2_blocking_callbacks to be set from 'Yes' to 'No'...
ophyd.utils.epics_pvs - Waiting for dg3_detector_image1_blocking_callbacks to be set from 'Yes' to 'No'...


TimeoutError: Failed to connect to SATT:FEE:320:COM:GO

In [ ]:
#Print Run Statistics
report = watcher.report()

Run Notes
------